In [1]:
pip install quandl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing, svm  #preprcessing scales data(the features). TO get the size of the data between -1 and 1
from sklearn import model_selection
# cross validation(model_selectionm) helps split the data
from sklearn.linear_model import LinearRegression

In [3]:
df = quandl.get("WIKI/AAPL")
df.head()

ConnectionError: HTTPSConnectionPool(host='www.quandl.com', port=443): Max retries exceeded with url: /api/v3/datasets/WIKI/AAPL/data?order=asc (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fcb103658d0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [21]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df["HL_PCT"] = (df["Adj. High"] - df['Adj. Close'])/df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open'])/ df['Adj. Open'] *100.0

df = df[['Adj. Close', 'HL_PCT', 'PCT_change', "Adj. Volume"]]

In [22]:
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
1980-12-12,0.422706,0.417391,0.000000,117258400.0
1980-12-15,0.400652,0.477064,-0.474799,43971200.0
1980-12-16,0.371246,0.475248,-0.473000,26432000.0
1980-12-17,0.380362,0.502513,0.000000,21610400.0
1980-12-18,0.391536,0.450620,0.000000,18362400.0


In [23]:
df.tail()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2018-03-21,171.270,2.230396,-2.153793,35247358.0
2018-03-22,168.845,2.271314,-0.679412,41051076.0
2018-03-23,164.940,3.019280,-2.048815,40248954.0
2018-03-26,172.770,0.191005,2.796454,36272617.0
2018-03-27,168.340,4.045384,-3.074620,38962839.0


In [24]:
# We will use the Adj Close as the feature we are trying to predict

In [40]:
forecastColumn = "Adj. Close"
df.fillna(-99999, inplace=True) #can't work with NA's in ML. We do this to treat the data as an outlier ratehr than delete everything


forecastOut = int(math.ceil(0.01 *len(df)))   #math.ceil rounds everything up to the nearest whole e.g 0.2 would round up to 1. Math.ceil returns a float, thus the int
# 0.1 because we want to predict 10% of the dataset


# creating labels

df["label"] = df[forecastColumn].shift(-forecastOut)
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,label
Date,,,,,,,,,,,,,
1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0,0.409914
1980-12-15,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.402563,0.402563,0.400652,0.400652,43971200.0,0.417266
1980-12-16,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.373010,0.373010,0.371246,0.371246,26432000.0,0.417266
1980-12-17,25.87,26.00,25.87,25.87,385900.0,0.0,1.0,0.380362,0.382273,0.380362,0.380362,21610400.0,0.415355
1980-12-18,26.63,26.75,26.63,26.63,327900.0,0.0,1.0,0.391536,0.393300,0.391536,0.391536,18362400.0,0.413443


In [51]:
# Features will be an X. Independent variables
# Lables will be a Y. Dependent variables

x = np.array(df.drop(["label"], 1)) #keep everything but the label column
y = np.array(df["label"])


x = preprocessing.scale(x)


y = np.array(df["label"])

xTrain, xTest,yTrain , yTest = model_selection.train_test_split(x, y, test_size = 0.2)

clf = LinearRegression() #n_jobs = -1) #clf = classifier   #can also use svm regression clf = svm.SVR() svm is server vector machine
# n_jobs determines how many threads you use . If you pick -1 which is equivalent to None, you use all threads
clf.fit(xTrain, yTrain)

accuracy = clf.score(xTest, yTest)

print(accuracy)
# svm accuracy = 0.9725823581120362 
# linear regression accuracy = 0.9748748069156078

0.974223191408065


/Users/wgl8723/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:173: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
